<h1>Import bibliotek</h1>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from langchain.chains import RetrievalQA
import torch
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 5.9 MB/s eta 0:00:00


<h1>Wczytanie PDFów</h1>

Do naszego modelu użyjemy PDFów zawierających informacje na temat ADHD.

In [6]:
loader = DirectoryLoader("/content/drive/MyDrive/MLkursADHD", glob="*.pdf", loader_cls=PyPDFLoader, show_progress=True)
pages = loader.load()
print(f"Wczytano {len(pages)} stron.")


100%|██████████| 3/3 [00:27<00:00,  9.10s/it]

Wczytano 372 stron.


Mamy 3 PDFy o liczbach stron: 11, 24 i 337, więc wszystkie strony zostały wczytane.

Przygotowujemy jeszcze dokument do dalszego przetwarzania.

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)
docs = text_splitter.split_documents(pages)

for idx,doc in enumerate(docs):
    print(f"{idx}.\n{doc}\n\n")
    if idx==2: break

0.
page_content='© COPYRIGHT 2004 PHYSICIANS POSTGRADUATE PRESS, INC. © COPYRIGHT 2004 PHYSICIANS POSTGRADUATE PRESS, INC.
A Guide to the Treatment of Adults With ADHD
27J Clin Psychiatry 2004;65 (suppl 3)
ttention-deficit/hyperactivity disorder (ADHD) is
known to affect approximately 4% to 12% of chil-' metadata={'producer': 'Acrobat Distiller 5.00 for Macintosh', 'creator': 'Adobe PageMaker 6.0', 'creationdate': '2004-03-03T22:22:36+00:00', 'moddate': '2004-03-05T10:04:25-06:00', 'subject': '', 'title': 'A Guide to the Treatment of Adults With ADHD', 'author': '', 'keywords': '', 'source': '/content/drive/MyDrive/MLkursADHD/ADHD11.pdf', 'total_pages': 11, 'page': 0, 'page_label': '27'}


1.
page_content='ttention-deficit/hyperactivity disorder (ADHD) is
known to affect approximately 4% to 12% of chil-
From the University of British Columbia Medical Center
and the Division of Child Psychiatry, British Columbia’s
Children’s Hospital, Vancouver.
This article is derived from the roundtab

Wszystko działa.

<h1>Baza danych</h1>

<h3>Embeddingi:</h3>

Do embeddingu użyjemy modelu all-MiniLM-L6-v2 jako, że ma mniej wymiarów przestrzeni osadzeń i będzie działać szybciej, bo ten z notebooka tutorialowego mielił zbyt długo moje PDFy. Możemy stracić przez to na jakości, ale ciekawi mnie czy będzie to widoczne w użytku.

In [8]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

/tmp/ipython-input-1219835146.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<h3>Baza danych:</h3>

Baza została zrobiona w plikach i wrzucona na dysk bo inaczej zbyt długo mi kręciło dlatego teraz tylko ładujemy.

In [13]:
db = FAISS.load_local("/content/drive/MyDrive/faissDB", embeddings, allow_dangerous_deserialization=True)

<h4>Test:</h4>

In [14]:
query = "Differences in presentation of child and adult ADHD"
similar_docs = db.similarity_search(query, k=3)  #top 3 wyniki

for doc in similar_docs:
    print(doc.page_content[:300] + "...\n---")

OF CHILD AND ADULT ADHD
Child psychiatrists who begin treatment of adults with
ADHD may ask if there is anything different about the
presentation of the condition in adults. In adults, just as in
children, the symptoms of ADHD are described by the...
---
and identification by the participating experts. Details
of the clinical presentation of adults with ADHD were
further informed by clinical expertise. This consensus
statement is written for specialists in psychiatry but is
also intended to increase the understanding of the dis-...
---
Attention-Deficit/Hyperactivity Disorder (ADHD).
In addition, it reviews what is known about
the social relations between children with
ADHD and their parents and peers. Along the
way, parental adjustment and psychiatric disor-
ders are also discussed, as they have a clear...
---


Działa

<h1>Model</h1>

Użyjemy modelu google/flan-t5-base, ponieważ jest dobry do większych danych niż ten w tutorialu co może być przydatne w naszym przypadku.

In [15]:
model_type = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_type)
tokenizer = AutoTokenizer.from_pretrained(model_type)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [16]:
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1024
)
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cpu
/tmp/ipython-input-996878642.py:7: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [17]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

result = qa_chain.invoke("Differences in presentation of child and adult ADHD")
print(f"AI Assistant:\n {result['result']}")

AI Assistant:
 In adults, just as in children, the symptoms of ADHD are described by the and identification by the participating experts. Details of the clinical presentation of adults with ADHD were further informed by clinical expertise.


Odpowiedź jest bardzo ogólnikowa, ale sprawdzimy jeszcze jak będzie sobie radzić pprzy innych pytaniach.

<h1>Chat</h1>

In [23]:
print("- - - Rozpocząłeś chat - - -", "/n")
aUser = input("Chcesz zadać pytanie [Q] czy zakończyć czat [E]? ")
while True:
    while str(aUser).lower() not in ['q','e']:
        aUser = input("Wpisz poprawną odpowiedź: Q - question, E - exit ")

    if str(aUser).lower() == 'q':
        qUser = input("Pytanie: ")
        #działanie atyhalucynogenne też nie działało, jeszcze spróbuję to zrobić
        aChat = str(qa_chain.invoke(qUser)['result'])

        if ("I don't know").strip().lower() in aChat.strip().lower():
            #dodatkowe cytowanie źródła i fragmentu dokumentu, na którym bazowana była odpowiedź
            zChat = 'title' #jeszcze mi nie działa
            fChat = db.similarity_search(aUser, k=1)
            print("Źródła: ", zChat, "\n")
            print("Fragment, który najbardziej wpłynął na odpowiedź: ", fChat[:300], "/n")
            print("Odpowiedź: ", aChat, "\n")

        else:
            print(aChat)

    else:
        print("- - - Zakończyłeś chat - - -")
        break

    aUser = input("Chcesz zadać pytanie [Q] czy zakończyć czat [E]? ")

- - - Rozpocząłeś chat - - - /n
Chcesz zadać pytanie [Q] czy zakończyć czat [E]? Q
Pytanie: Differences in presentation of child and adult ADHD
In adults, just as in children, the symptoms of ADHD are described by the and identification by the participating experts. Details of the clinical presentation of adults with ADHD were further informed by clinical expertise.
Chcesz zadać pytanie [Q] czy zakończyć czat [E]? Q
Pytanie: is ADHD genetic
no
Chcesz zadać pytanie [Q] czy zakończyć czat [E]? Q
Pytanie: is ADHD more common among women or men
males
Chcesz zadać pytanie [Q] czy zakończyć czat [E]? 5 most common ADHD symptoms
Wpisz poprawną odpowiedź: Q - question, E - exit Q
Pytanie: 5 most common ADHD symptoms
mood, anxiety, sleep, conduct and By later childhood and preadolescence, these patterns of academic, familial, and social con - flicts have become well established for many children with ADHD. At least 40–85% have developed ODD (see Chapter 4), and as many as 25–50% are likely to d

<h1>Wnioski</h1>

Zauważalnymi problemami modelu są:
* odpowiedzi zawierające mało wartościowe informacje - dłuższe odpowiedzi nie wnoszą faktycznych informacji
* niektóre słowa są rozbite na 2 człony tak jakby były przenoszone do kolejnej linii (np. "con - flicts")
* nie udało mi się jeszcze zrobić leku na halucynacje więc przy pytaniu typu "do I have ADHD" model zmyśla (unless...?)

Dodatkowo trenowanie jest bardzo wymagające zasobowo - przy próbie trenowania na 800 stronach (przed okrojeniem największego dokumentu do 300 stron) poddałam się po 1.5h ładowania, chociaż też akurat wtedy nie miałam dostępu do zbyt dobrego internetu, więc może w innych warunkach byłoby to wykonalne.